In [1]:
1

1

In [2]:
(start-swank)

T

In [3]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/addions/")

In [4]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [5]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [6]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [7]:
(:= *ff* (leap.core::merged-force-field))

*FF*

In [8]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [9]:
(build-hydrogens *chemagg*)

NIL

In [10]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

In [11]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C      S


NIL

In [12]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [13]:
(cando:jostle *chemagg*)

NIL

In [14]:
(defparameter *ee* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minSDnP       0     1      -inf   0.0              0.000          17375.429
 minSDnP       0     2     -7.99   0.0        2649742.383          10019.048
 minSDnP       0     3     -8.30   0.0        2197521.753           5633.562
DONE absolute force test:
forceRmsMag(4051.371576).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     4      -inf   0.0              0.000           4051.372
 minCGnP       0     5     -7.74  38.2        1457984.565           3192.634
 minCGnP       0     6     -6.67  45.7        1031445.839           2635.969
 minCGnP       0     7     -7.70  56.9         914707.921           2900.914
 minCGnP       0     8     -8.17  49.5         865958.013           2598.774
 minCGnP       0     9     -7.37  82.1         711156.715          12182.936
 minCGnP       0    10    -11.73  17.1   

*EE*

In [15]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

In [16]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [17]:
(defparameter *mol1* (chem:make-molecule))
(defparameter *res1* (chem:make-residue :foo))
(defparameter *a1* (chem:make-atom :CL :|Cl|))
(chem:set-position *a1* '(0.0 0.0 0.0))
(chem:set-charge *a1* '-1.0)
(chem:add-matter *res1* *a1*)
(chem:add-matter *mol1* *res1*)
(defparameter *clion* (chem:make-aggregate))
(chem:add-matter *clion* *mol1*)

NIL

In [18]:
(defparameter *o1* (core:make-cxx-object 'chem:octree))

*O1*

In [19]:
(defparameter *ot* (chem:oct-oct-tree-create *o1* *chemagg* 1 1.0 1.0 4.0 0 t))

*OT*

In [20]:
(defparameter *newpos* (chem:oct-tree-init-charges *o1* *ot* 1 1 1.5))

*NEWPOS*

In [21]:
(defparameter *clion-copy* (chem:matter-copy *clion*))

*CLION-COPY*

In [22]:
(defparameter *clion-transform* (geom:make-m4-translate *newpos*))

*CLION-TRANSFORM*

In [23]:
(chem:apply-transform-to-atoms *clion-copy* *clion-transform*)

NIL

In [24]:
(chem:add-matter *chemagg* *clion-copy*)

NIL

In [25]:
(defparameter *ot* (chem:oct-oct-tree-create *o1* *chemagg* 1 1.0 1.0 4.0 0 t))

*OT*

In [26]:
(defparameter *newpos1* (chem:oct-tree-init-charges *o1* *ot* 1 1 1.5))

*NEWPOS1*

In [ ]:
(chem:oct-tree-delete-sphere *o1* *ot* *newpos* 1.5)

In [ ]:
(chem:oct-tree-update-charge *o1* *ot* *newpos* -1.0 1.5)

In [17]:
(save-mol2 *chemagg* "alanine.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/addions/alanine.mol2


NIL

In [16]:
(defparameter *chemagg2* (chem:matter-copy *chemagg*))

*CHEMAGG2*

In [17]:
(defparameter *chemcenter* (chem:geometric-center *chemagg2*))

*CHEMCENTER*

In [18]:
(defparameter *transform* (geom:make-m4-translate (geom:v* *chemcenter* -1.0)))

*TRANSFORM*

In [19]:
(chem:apply-transform-to-atoms *chemagg2* *transform*))

NIL

In [17]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

In [18]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [19]:
(nglv::show-aggregate *solvent*)

SIMPLE-WARNING: 
Finish %set-coordinates
SIMPLE-WARNING: 
Finish %handle-n-components-changed


A Jupyter Widget

In [20]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [21]:
(:= *newagg* (leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0)))

Total bounding box for atom centers:  29.504539513834402d0 24.43666854312727d0 24.18439487940269d0


*NEWAGG*

In [22]:
(nglv::show-aggregate *newagg*)

SIMPLE-WARNING: 
Finish %set-coordinates
SIMPLE-WARNING: 
Finish %handle-n-components-changed


A Jupyter Widget

In [23]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [8]:
(defparameter *chemleap* (cando:load-mol2 "alanine-leap.mol2"))

*CHEMLEAP*

In [9]:
(nglv::show-aggregate *chemleap*)

A Jupyter Widget

In [10]:
(defparameter *oleap* (core:make-cxx-object 'chem:octree))

*OLEAP*

In [14]:
(defparameter *otl* (chem:oct-oct-tree-create *oleap* *chemleap* 1 1.0 1.5 4.0 0 t))

*OTL*

In [19]:
(defparameter *newposl* (chem:oct-tree-init-charges *oleap* *otl* 1 1 1.5))

*NEWPOSL*

In [20]:
*newposl*

[ -2.51374 -3 -2.1312 ]

In [37]:
(defparameter *mol1* (chem:make-molecule))
(defparameter *res1* (chem:make-residue :foo))
(defparameter *a1* (chem:make-atom :CL :|Cl|))
(chem:set-position *a1* '(0.0 0.0 0.0))
(chem:set-charge *a1* '-1.0)
(chem:add-matter *res1* *a1*)
(chem:add-matter *mol1* *res1*)
(defparameter *clion* (chem:make-aggregate))
(chem:add-matter *clion* *mol1*)
(defparameter *clion-copy1* (chem:matter-copy *clion*))
(defparameter *clion-transform* (geom:make-m4-translate *newposl*))
(chem:apply-transform-to-atoms *clion-copy* *clion-transform*)
(chem:add-matter *chemleap* *clion-copy*)

NIL

In [38]:
(defparameter *otl* (chem:oct-oct-tree-create *oleap* *chemleap* 1 1.0 1.0 4.0 0 t))

*OTL*

In [39]:
(defparameter *newposl* (chem:oct-tree-init-charges *oleap* *otl* 1 1 1.5))

*NEWPOSL*

In [40]:
*newposl*

[ -0.013741 1.5 -6.85722 ]